FOR PAGE 1

In [ ]:
import pandas as pd
import os

# ---------------------------------------------------------
# CONFIGURATION
# ---------------------------------------------------------
INPUT_PATH = "olim_2015-2024_preprocessed.csv" # Your raw 'single sightings' file
OUTPUT_PATH = "olim_aggregated.csv"             # The new optimized file

def create_aggregated_dataset():
    print(f"Loading raw data from {INPUT_PATH}...")

    try:
        df = pd.read_csv(INPUT_PATH)
    except FileNotFoundError:
        print("Error: Input file not found. Please check the path.")
        return

    original_rows = len(df)
    print(f"Original row count: {original_rows:,}")

    # ---------------------------------------------------------
    # 1. CLEANING (Logic copied from your Streamlit app)
    # ---------------------------------------------------------
    print("Cleaning data...")

    # Standardize country names
    if "erez_moza" in df.columns:
        df["erez_moza"] = df["erez_moza"].astype(str).str.strip()

    # Ensure numeric years/months
    df["year_aliya"] = pd.to_numeric(df["year_aliya"], errors='coerce')
    df["month_aliya"] = pd.to_numeric(df["month_aliya"], errors='coerce')
    df = df.dropna(subset=["year_aliya", "month_aliya"])

    # Create Date column
    df["date"] = pd.to_datetime(
        df["year_aliya"].astype(int).astype(str) + "-" +
        df["month_aliya"].astype(int).astype(str) + "-01",
        errors="coerce"
    )
    df = df.dropna(subset=["date"])

    # ---------------------------------------------------------
    # 2. AGGREGATION
    # ---------------------------------------------------------
    print("Aggregating data...")

    # Group by Date, Country, and Continent to get counts
    # We maintain 'continent' here so we don't lose it in the app
    aggregated_df = (
        df.groupby(["date", "erez_moza", "continent"], as_index=False)
        .size()
        .rename(columns={"size": "monthly_count"})
    )

    # Sort for cleaner usage later
    aggregated_df = aggregated_df.sort_values(by=["date", "erez_moza"])

    # ---------------------------------------------------------
    # 3. SAVING
    # ---------------------------------------------------------
    new_rows = len(aggregated_df)
    reduction = (1 - (new_rows / original_rows)) * 100

    print(f"Aggregated row count: {new_rows:,}")
    print(f"Data reduction: {reduction:.2f}%")

    aggregated_df.to_csv(OUTPUT_PATH, index=False)
    print(f"File saved successfully to: {OUTPUT_PATH}")

if __name__ == "__main__":
    create_aggregated_dataset()

In [ ]:
import pandas as pd
import numpy as np

# --- CONFIGURATION: UPDATE THESE PATHS ---
PATH_OLIM = "/content/olim_aggregated.csv"
PATH_GDP = "/content/gdp.csv"
OUTPUT_FILENAME = "page1_final.csv"

def create_aggregated_file():
    print("Loading data...")

    # 1. Load and Clean Olim Data
    try:
        # Assuming olim file has columns: date, erez_moza, monthly_count, continent
        df_olim = pd.read_csv(PATH_OLIM, parse_dates=["date"])
        # Ensure date is normalized to start of month just in case
        df_olim["date"] = df_olim["date"] + pd.offsets.MonthBegin(0)
    except Exception as e:
        print(f"Error loading Olim data: {e}")
        return

    # 2. Load and Process GDP Data
    try:
        df_gdp_raw = pd.read_csv(PATH_GDP)
    except Exception as e:
        print(f"Error loading GDP data: {e}")
        return

    # Clean GDP
    if "erez_moza" in df_gdp_raw.columns:
        df_gdp_raw = df_gdp_raw.dropna(subset=["erez_moza"])
        df_gdp_raw["erez_moza"] = df_gdp_raw["erez_moza"].astype(str).str.strip()

    # Extract Country Name Mapping (Hebrew -> English)
    # We will merge this back later to ensure every row has the English name
    hebrew_to_english_df = df_gdp_raw[["erez_moza", "Country"]].dropna().drop_duplicates(subset=["erez_moza"])

    # Melt GDP (Years to Rows)
    year_cols = [c for c in df_gdp_raw.columns if c.isdigit()]
    gdp_melt = df_gdp_raw.melt(
        id_vars=["erez_moza"],
        value_vars=year_cols,
        var_name="year",
        value_name="gdp"
    )
    gdp_melt["year"] = pd.to_numeric(gdp_melt["year"])
    gdp_melt["gdp"] = pd.to_numeric(gdp_melt["gdp"], errors='coerce')

    # Create Date column (Jan 1st of every year)
    gdp_melt["date"] = pd.to_datetime(gdp_melt["year"].astype(str) + "-01-01")
    gdp_melt = gdp_melt.dropna(subset=["gdp", "date"])
    gdp_melt = gdp_melt.sort_values(["erez_moza", "date"])

    # Interpolate GDP to Monthly level
    print("Interpolating GDP data (this might take a moment)...")
    gdp_interp = (
        gdp_melt.set_index("date")
        .groupby("erez_moza")["gdp"]
        .apply(lambda x: x.resample("MS").interpolate(method="linear"))
        .reset_index()
    )

    # 3. Merge Datasets
    # We use an outer join to keep GDP data even if no immigration happened that month, and vice versa
    print("Merging datasets...")
    df_merged = pd.merge(
        df_olim,
        gdp_interp,
        on=["date", "erez_moza"],
        how="outer"
    )

    # 4. Cleanup Merged Data
    # Fill NaNs for monthly_count with 0 (if a month exists in GDP but not in Olim, count is 0)
    df_merged["monthly_count"] = df_merged["monthly_count"].fillna(0)

    # Forward fill Continents and English Names per country
    # (Because rows coming purely from GDP might miss the continent info from the Olim file)

    # First, merge the English names we extracted earlier
    df_merged = pd.merge(df_merged, hebrew_to_english_df, on="erez_moza", how="left")

    # Now fix missing continents. We group by country and fill forward/backward
    if "continent" in df_merged.columns:
        df_merged["continent"] = df_merged.groupby("erez_moza")["continent"].transform(lambda x: x.ffill().bfill())

    # Drop rows where we still don't have a continent (usually dirty data) or Date
    df_merged = df_merged.dropna(subset=["date", "continent"])

    # --- NEW: HARD CUTOFF FOR DATE ---
    # This ensures we don't get January 2025 just because of GDP interpolation
    df_merged = df_merged[df_merged["date"] <= "2024-12-01"]
    # ---------------------------------

    # Sort
    df_merged = df_merged.sort_values(by=["erez_moza", "date"])

    # 5. Save
    print(f"Saving to {OUTPUT_FILENAME}...")
    df_merged.to_csv(OUTPUT_FILENAME, index=False)
    print("Done! You can now upload this file.")

if __name__ == "__main__":
    create_aggregated_file()

FOR PAGE 2

In [ ]:
import pandas as pd
import numpy as np

def create_aggregated_dataset():
    print("--- 1. Loading Data ---")
    try:
        # Load the Raw Olim Data
        df_raw = pd.read_csv("olim_2015-2024_preprocessed.csv")

        # Load the Mapping File
        df_map = pd.read_csv("mapped_yeshuvim.csv")
        df_map['mapped_district'] = df_map['mapped_district'].astype(str).str.strip()

        # Load the Scores File
        df_scores = pd.read_csv("isr_data.csv")
        df_scores['english'] = df_scores['english'].astype(str).str.strip()
        # Convert scores to numeric, forcing errors to NaN then filling with 0
        df_scores['score'] = pd.to_numeric(df_scores['score'], errors='coerce').fillna(0)

    except FileNotFoundError as e:
        print(f"Error: Could not find file {e.filename}")
        return

    print("--- 2. Preprocessing Raw Data ---")

    # A. Employment Status Logic
    # subject: 'לא עבד' or 'לא צויין' -> Not Employed (0). Others -> Employed (1)
    if 'subject' in df_raw.columns:
        df_raw['is_employed'] = (~df_raw['subject'].isin(['לא עבד', 'לא צויין'])).astype(int)
    else:
        print("Warning: 'subject' column missing. simulating employment data.")
        df_raw['is_employed'] = np.random.randint(0, 2, size=len(df_raw))

    # B. Age Logic
    # Calculate Age from Birth Year (assuming current year 2025)
    if 'age' not in df_raw.columns:
        if 'birth_year' in df_raw.columns:
            df_raw['age'] = 2025 - df_raw['birth_year']
        else:
            print("Warning: 'age' and 'birth_year' missing. Simulating ages.")
            df_raw['age'] = np.random.randint(18, 90, size=len(df_raw))

    # C. Gender Logic (Convert to numeric for averaging)
    # 1 = Female, 0 = Male
    if 'gender' in df_raw.columns:
        df_raw['is_female'] = (df_raw['gender'] == 'נקבה').astype(int)
    else:
        df_raw['is_female'] = 0.5 # Fallback

    print("--- 3. Aggregating Data ---")

    # Group by Hebrew Name ('yeshuv_klita')
    # We calculate: Count (Total), Mean Age, Mean Employment, Mean Female %
    df_agg = df_raw.groupby('yeshuv_klita').agg(
        total_olim=('age', 'count'),
        avg_age=('age', 'mean'),
        pct_employed=('is_employed', 'mean'),
        pct_female=('is_female', 'mean')
    ).reset_index()

    # Convert fractions (0.5) to Percentages (50.0)
    df_agg['pct_employed'] = (df_agg['pct_employed'] * 100).round(1)
    df_agg['pct_female'] = (df_agg['pct_female'] * 100).round(1)
    df_agg['pct_male'] = (100 - df_agg['pct_female']).round(1)
    df_agg['avg_age'] = df_agg['avg_age'].round(1)

    print("--- 4. Merging with Metadata (IDs and Scores) ---")

    # Merge 1: Add English IDs from mapping file
    # We join on the Hebrew name
    df_merged = df_agg.merge(
        df_map[['hebrew_name', 'mapped_district']],
        left_on='yeshuv_klita',
        right_on='hebrew_name',
        how='inner' # Keep only cities we have a map for
    )

    # Merge 2: Add Scores from isr_data
    # We join on the English ID ('mapped_district' vs 'english')
    df_final = df_merged.merge(
        df_scores[['english', 'score']],
        left_on='mapped_district',
        right_on='english',
        how='left'
    )

    # Rename columns for clarity in the app
    df_final = df_final.rename(columns={
        'mapped_district': 'english_id',
        'hebrew_name': 'hebrew_name'
    })

    # Fill missing scores with 0
    df_final['score'] = df_final['score'].fillna(0)

    # Clean up columns (drop duplicates/helper cols)
    cols_to_keep = ['english_id', 'hebrew_name', 'total_olim', 'avg_age', 'pct_employed', 'pct_female', 'pct_male', 'score']
    df_final = df_final[cols_to_keep]

    print(f"--- 5. Saving to CSV ({len(df_final)} cities) ---")
    df_final.to_csv("olim_aggregated_ready.csv", index=False)
    print("Done! File 'olim_aggregated_ready.csv' created successfully.")

if __name__ == "__main__":
    create_aggregated_dataset()

In [ ]:
import pandas as pd

# 1. Load the datasets
olim_df = pd.read_csv('olim_aggregated_ready.csv')
isr_df = pd.read_csv('isr_data.csv')

# 2. Merge the dataframes
# We map 'english_id' from the olim dataset to the 'english' column in the israel data
merged_df = olim_df.merge(
    isr_df[['english', 'madad']],
    left_on='english_id',
    right_on='english',
    how='left'
)

# 3. Clean up (remove the duplicate 'english' column)
merged_df = merged_df.drop(columns=['english'])

# 4. Save the result
merged_df.to_csv('page2_final.csv', index=False)

FOR PAGE 3

In [ ]:
import pandas as pd

# 1. Setup Paths
PATH_RAW_DATA = '/content/olim_2015-2024_preprocessed.csv'
OUTPUT_PATH = 'page3_final.csv'

def process_and_save_data():
    print("Loading raw data...")
    df = pd.read_csv(PATH_RAW_DATA)

    # 2. Aggregation / Mapping
    # (Applying this first, exactly as in your original script)
    mapping = {
        'מדעים מדויקים': 'טכנולוגיה והנדסה',
        'מקצועות המחשב': 'טכנולוגיה והנדסה',
        'חקלאות ובעלי חיים': 'מדעי החיים'
    }
    df['subject'] = df['subject'].replace(mapping)

    # 3. Exclusions
    # Note: 'חקלאות ובעלי חיים' was mapped to 'מדעי החיים' above.
    # Since 'מדעי החיים' is in the exclusion list, those rows will be removed.
    exclusions = ["בלטי מקצועי", "בלתי מקצועי", "לא עבד", "לא צויין", "עצמאי", "מדעי החיים"]
    df_clean = df[~df['subject'].isin(exclusions)]

    # 4. Grouping (The Aggregation Step)
    # We count how many people moved from specific Country -> Subject
    df_aggregated = df_clean.groupby(['erez_moza', 'subject']).size().reset_index(name='count')

    # 5. Save
    print(f"Original rows: {len(df)}")
    print(f"Aggregated rows: {len(df_aggregated)}")
    df_aggregated.to_csv(OUTPUT_PATH, index=False)
    print(f"Saved aggregated data to {OUTPUT_PATH}")

if __name__ == "__main__":
    process_and_save_data()